## Woermann Brock 

Serves as a normalization script before the execution of the preload.

In [1]:
import pandas as pd
import re
import datetime
import os

from datetime import datetime

In [2]:
to_upload_folder_path = r'R:\RawData\Elite Star\\Woermann Brock\ToUpload'
done_folder_path = r'R:\RawData\Elite Star\\Woermann Brock\\Done'

In [3]:
# Returns the latest file in the specified path based on the date modified
def get_latest_file(path):
  files = os.listdir(path)
  latest_file = max(files, key=lambda x: os.path.getmtime(os.path.join(path, x)))
  return latest_file

In [4]:
def extract_date_from_filename(filename):
    match = re.search(r'(\d{2})(\d{2})(\d{4})', filename)
    if match:
        date_str = match.group(1) + match.group(2) + match.group(3)
        return datetime.strptime(date_str, '%d%m%Y')
    else:
        return datetime.min # If the date is not found, set it to a default value

file_list = os.listdir(to_upload_folder_path)
file_list.sort(key=lambda x: extract_date_from_filename(x), reverse=True)

last_file_processed = get_latest_file(done_folder_path)

sorted_files = [{'File Name': filename, 'Date': extract_date_from_filename(filename), 'Last File Processed': last_file_processed} for filename in file_list]

df = pd.DataFrame(sorted_files)
print(df)

                File Name       Date      Last File Processed
0  ESTexport-25022025.txt 2025-02-25  ESTexport-11022025 .txt
1  ESTexport-13022025.txt 2025-02-13  ESTexport-11022025 .txt
2  ESTexport-12022025.txt 2025-02-12  ESTexport-11022025 .txt
3  ESTexport-11022025.txt 2025-02-11  ESTexport-11022025 .txt
4  ESTexport-10022025.txt 2025-02-10  ESTexport-11022025 .txt
5  ESTexport-09022025.txt 2025-02-09  ESTexport-11022025 .txt
6  ESTexport-08022025.txt 2025-02-08  ESTexport-11022025 .txt
7  ESTexport-07022025.txt 2025-02-07  ESTexport-11022025 .txt
8  ESTexport-06022025.txt 2025-02-06  ESTexport-11022025 .txt


In [5]:
def normalize_filename(filename):
    base, ext = os.path.splitext(filename)
    normalized_filename = f"{base.strip()} {ext.strip()}"
    return normalized_filename

file_data = []
for filename in os.listdir(to_upload_folder_path):
    if os.path.isfile(os.path.join(to_upload_folder_path, filename)):
        normalized_filename = normalize_filename(filename)
        if normalized_filename != filename:  
            file_data.append([filename, normalized_filename])
            destination_path = os.path.join(to_upload_folder_path, normalized_filename)
            if os.path.exists(destination_path):
                os.remove(destination_path)  # Remove the existing file
            os.rename(os.path.join(to_upload_folder_path, filename), destination_path)

df = pd.DataFrame(file_data, columns=['Initial Filename', 'Final Filename'])
print(df)

         Initial Filename           Final Filename
0  ESTexport-06022025.txt  ESTexport-06022025 .txt
1  ESTexport-07022025.txt  ESTexport-07022025 .txt
2  ESTexport-08022025.txt  ESTexport-08022025 .txt
3  ESTexport-09022025.txt  ESTexport-09022025 .txt
4  ESTexport-10022025.txt  ESTexport-10022025 .txt
5  ESTexport-11022025.txt  ESTexport-11022025 .txt
6  ESTexport-12022025.txt  ESTexport-12022025 .txt
7  ESTexport-13022025.txt  ESTexport-13022025 .txt
8  ESTexport-25022025.txt  ESTexport-25022025 .txt
